In [1]:
import re

class Token:
    def __init__(self, token_type, value):
        self.token_type = token_type
        self.value = value

def lexical_analyzer_music(input_string):
    tokens = []
    if not input_string.strip():
        return tokens  # Return empty list if input string is empty or contains only whitespace

    # Define regular expressions for token types
    patterns = [
        (r'C|D|E|F|G|A|B', 'NOTE'),
        (r'(1|2|4|8)', 'DURATION'),
        (r'play\s+[A-Za-z]+', 'PLAY'),
        (r'unstop', 'UNSTOP'),
        (r'help', 'HELP'),
        (r'R', 'REST'),
        (r'print','PRINT')
    ]
    
    lines = input_string.strip().split('\n')  # Split input into lines
    for line in lines:
        # Remove comments starting with $
        line = re.sub(r'\$.+', '', line)
        line = line.strip()  # Remove leading and trailing whitespace
        if not line:
            continue  # Skip empty lines
        
        # Process each line
        while line:
            matched = False
            for pattern, token_type in patterns:
                match = re.match(pattern, line)
                if match:
                    if token_type == 'PLAY':
                        name = match.group().split("play")[1].strip()
                        tokens.append(Token(token_type, name))
                    else:
                        tokens.append(Token(token_type, match.group()))
                    line = line[len(match.group()):].lstrip()
                    matched = True
                    break
            
            if not matched:
                print("\033[91m" + "Error: Unknown token - " + line[0] + "\033[0m")
                return []  # Return empty list if unknown token encountered
    return tokens

# Example usage
music_code = """
print
"""
tokens = lexical_analyzer_music(music_code)
if tokens:
    for token in tokens:
        print(token.token_type, token.value)
else:
    print("No tokens generated due to error.")


PRINT print


In [2]:
class MusicParser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.token_index = 0
        self.parse_tree = []

    def parse(self):
        self.command_list()

    def command_list(self):
        self.command()
        while self.token_index < len(self.tokens):
            self.command()

    def command(self):
        if self.token_index < len(self.tokens):
            current_token = self.tokens[self.token_index]
            if current_token.token_type == 'NOTE':
                self.note()
                

            elif current_token.token_type == 'PLAY':
                self.play()
            elif current_token.token_type == 'UNSTOP':
                self.unstop()
            elif current_token.token_type == 'HELP':
                self.help()
            elif current_token.token_type == 'REST':
                self.rest()
            elif current_token.token_type == 'PRINT':
                self.print_help()
            else:
                print("\033[91m" + "Error: Unexpected token - " + current_token.value + "\033[0m")
                exit(1)

    def note(self):
    # Add the note to the parse tree
        self.parse_tree.append("Note: " + self.tokens[self.token_index].value)
        self.token_index += 1
    
    # Check if the next token exists and is a DURATION token
        if self.token_index < len(self.tokens):
            next_token = self.tokens[self.token_index]
            if next_token.token_type != 'DURATION':
                print("\033[91m" + "Error: Expected DURATION token after NOTE - " + next_token.value + "\033[0m")
                exit(1)
            else:
            # Increment the token index to move to the next token
                self.parse_tree.append("Duration: " + self.tokens[self.token_index].value)
                self.token_index += 1
        else:
        # If there are no more tokens, exit parsing
            return
   



    def play(self):
       
        self.parse_tree.append("Play: " + self.tokens[self.token_index].value)
        self.token_index += 1

    def unstop(self):
        self.parse_tree.append("Unstop")
        self.token_index += 1

    def help(self):
        self.parse_tree.append("Help")
        self.token_index += 1
    
    def print_help(self):
        self.parse_tree.append("Print")
        self.token_index += 1

    def rest(self):
        self.parse_tree.append("Rest")
        self.token_index += 1


In [4]:
import sounddevice as sd
import numpy as np
import pyttsx3
import time
import pygame

class MusicInterpreter:
    def __init__(self, parse_tree):
        self.parse_tree = parse_tree
        self.notes = {
            "C": 261.63, "D": 393.66, "E": 429.63,
            "F": 549.23, "G": 692.00, "A": 740.00, "B": 893.88
        }
        
#            "C": 261.63, "D": 293.66, "E": 329.63,
#             "F": 349.23, "G": 392.00, "A": 440.00, "B": 493.88
        self.durations = {
            
            "1": 1, "2": 2, "4": 4, "8": 8
            # Define durations for different note lengths
        }


    def interpret(self):
     
        sample_rate = 44100  # Sample rate (samples per second)
        frequency = None  # Initialize frequency

        for element in self.parse_tree:
            if "Note" in element:
              
                frequency = self.notes.get(element.split('Note: ')[-1])
            elif "Duration" in element:
             
                
                duration = self.durations.get(element.split(': ')[1])
               
                if frequency and duration:
                    self.play_frequency(frequency, duration)
            elif "Rest" in element:
                self.wait(2)
            elif "Play" in element:
                name = element.split('PLAY: ')[-1].split(' ')[1:]
                self.play(name)
            elif "Help" in element:
                self.help()
            elif "Print" in element:
                self.print_music_code_manual()
            elif "Unstop" in element:
                self.play_unstoppable()

        sd.stop()

    def play_frequency(self, frequency,duration):
        sample_rate = 44100
#         duration = 0.5
        t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
        waveform = np.sin(2 * np.pi * frequency * t)
        sd.play(waveform, samplerate=sample_rate)
        sd.wait()

    def wait(self, duration):
        time.sleep(duration)

    def play(self,name):
        
          # Extract the name value
        
        
        name=name[0]
       
        # Initialize Pygame
        pygame.init()

        # Define a mapping of letters to musical notes (you can define your own mapping)
        # Frequencies for all letters (A to Z)
        notes = {
            'A': 440.00,
            'B': 493.88,
            'C': 523.25,
            'D': 587.33,
            'E': 659.25,
            'F': 698.46,
            'G': 783.99,
            'H': 880.00,
            'I': 987.77,
            'J': 1046.50,
            'K': 1174.66,
            'L': 1318.51,
            'M': 1396.91,
            'N': 1567.98,
            'O': 1760.00,
            'P': 1975.53,
            'Q': 2093.00,
            'R': 2349.32,
            'S': 2637.02,
            'T': 2793.83,
            'U': 3135.96,
            'V': 3520.00,
            'W': 3951.07,
            'X': 4186.01,
            'Y': 4698.63,
            'Z': 5274.04,
            ' ': 0  # Represent space as 0 frequency (rest)
        }

        # Convert name to a sequence of musical notes
        melody = [notes[letter.upper()] for letter in name if letter.upper() in notes]

        # Define a function to create a sound for a specific frequency and duration
        def create_sound(freq, duration):
            sample_rate = 44100  # Change this value for different audio qualities
            num_samples = int(sample_rate * duration)
            volume = 0.5  # Adjust the volume (0.0 to 1.0)

            times = np.linspace(0, duration, num_samples, endpoint=False)
            waveform = volume * (32767 * np.sin(2 * np.pi * freq * times)).astype(np.int16)
            
            
            
            
            sound = pygame.mixer.Sound(buffer=waveform)
            return sound

        # Play the melody
        for freq in melody:
            if freq != 0:  # Play note if not a rest
                sound = create_sound(freq, 0.5)  # Duration of 0.5 seconds
                sound.play()
        
            time.sleep(0.5)  # Adjust this to control the speed of the melody

        # Quit Pygame
        pygame.quit()

    def help(self):
        engine = pyttsx3.init()
        
        
        engine.say('Welcome to Raag. This musical language was created by Shreya Gandhi. '  
        'It allows you to play notes from A to G and specify durations of 1, 2, 4, 8, or 16. '
        'Raag features commands such as "unstop" to play unstoppable music, '
        'help command is used for accessing vocal manual. '
        'print command is used for displaying manual on the console. '
        'rest command is used to introduce pauses, and play command is used to vocalize command descriptions. '
        'You can use tab space or newline as a separator. '
        '$ is used for single line comment. Thank you and keep Coding !')
        engine.runAndWait()

    def play_unstoppable(self):
        pygame.init()
        pygame.mixer.music.load(r'G:\lt_lab\raag\Raag\unstoppable.mp3')
        pygame.mixer.music.play(0)
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)
        pygame.quit()
    def print_music_code_manual(self):
        manual = """
        
        ________                              
        `MMMMMMMb.                            
         MM    `Mb                            
         MM     MM    ___      ___     __     
         MM     MM  6MMMMb   6MMMMb   6MMbMMM 
         MM    .M9 8M'  `Mb 8M'  `Mb 6M'`Mb   
         MMMMMMM9'     ,oMM     ,oMM MM  MM   
         MM  \M\   ,6MM9'MM ,6MM9'MM YM.,M9   
         MM   \M\  MM'   MM MM'   MM  YMM9    
         MM    \M\ MM.  ,MM MM.  ,MM (M       
        _MM_    \M\`YMMM9'Yb`YMMM9'Yb.YMMMMb. 
                                     6M    Yb 
                                     YM.   d9 
                                      YMMMM9  

        Manual for Raag by Shreya Gandhi:
    
        A to G followed by duration (1, 2, 4, 8) seconds respectively.
    
        Code can be separated by newline or tab space.
    
        $ is used for single-line comments.
    
        'R' - Rest is used to give a pause for 2 seconds.
    
        'play shreya' command will play the frequency based on the alphabets
         present in the word 'shreya'.(Result will be same for words in capital or small letters)
        
        'unstop' will play unstoppable music.
        
        'help' is used for vocal manual help.
        
        'print is used for displaying manual'
    
        Try out this example:
        
        C1 D1 G1 A1 B1 F1 E1 $ This is a comment
        play shreya C2
        D8 $ Another comment E2
        R help
        E2
        unstop
        print
        
        """
        print(manual)


# Tokenize the music code
print("Please enter your music code below (press Enter twice to finish):")
print("───────────────────────────────────────────────────────────────────────")
music_code_lines = []
while True:
    line = input("> ")
    if not line.strip():  # If the line is empty (user pressed Enter)
        break
    music_code_lines.append(line)

    # Combine the lines into a single multi-line string
music_code = "\n".join(music_code_lines)

tokens = lexical_analyzer_music(music_code)

# Parse the tokens
parser = MusicParser(tokens)
parser.parse()

interpreter = MusicInterpreter(parser.parse_tree)
interpreter.interpret()


# can try these commands
#help    C1 D1 G1 A1 B1 F1 E1   play shreya R  C2  unstop print $THIS IS A COMMENT

Please enter your music code below (press Enter twice to finish):
───────────────────────────────────────────────────────────────────────
> help
> 
